### This notebook serves as a template for other notebooks, copy it to a new file and work from there rather than modifying this notebook.

In [ ]:
# generally useful imports
import pandas as pd
import numpy as np

# Mount data either from disk or bucket
from pathlib import Path
home = str(Path.home())
try:
    data_dir = "/mnt/disks/lcs/tess-goddard-lcs/"  # for disk
except:
    data_dir = home + "/mountpoint/"  # for bucket

# Add spocc directory to the path to import spocc modules
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import data
import loaders

%load_ext autoreload
%autoreload 2

In [ ]:
# Make sure you can load data
try:
    # check if the lookup file for sector one exists
    ref = pd.read_csv(data_dir+"sector1lookup.csv")
except FileNotFoundError:
    # If not, try to mount it.
    #! The following line needs to be changed to match the data_dir
    os.system(
        f"gcsfuse --implicit-dirs tess-goddard-lcs {data_dir}"
    )
    ref = pd.read_csv(data_dir+"sector1lookup.csv")
    
ref.head()

# Load lookup table

In [ ]:
## Loading in the skyportal lookup table, constrained to the most anomalous from each sector
ref = pd.read_csv("/home/jupyter/skyportal_lookup.csv", index_col=0)
## Loading in a lookup table for a specific sector
# s = 1
# ref = pd.read_csv(data_dir+f"sector{s}lookup.csv")

## Create reference table of interest

In [ ]:
# Change this cell to create a reference dataframe with targets of interest
## Example for a given sector/camera
i = j = 1
subref = ref[(ref.Sector==i)&(ref.Camera==j)] 

## Example for specific TICs in a particular sector
# tics = [124106762, 52640302, 147314529, 287326187, 370101492, 93420238, 35718753, 34442932]
# subref = ref[(ref.Sector==i)&(ref.TIC_ID.isin(tics)]

# Load light curves

## Individual light curve

In [ ]:
# this loads lightcurves without MAD cuts or quality filters
f0 = data_dir+subref.Filename.values[0]
lc = loaders.load_lc(f0)

## Set of light curves

In [ ]:
def load_subref_lcs(subref):
    if subref.shape[0]>500:
        subref = subref.iloc[:500]

    files = data_dir+subref.Filename.values

    sec = subref.Sector.values
    cam = subref.Camera.values
    # this will load light curves with MAD cuts and quality filters by default
    lcc = loaders.LightCurveCollection(sec, cam, files)

    lcc.load_all_lcs()

    return lcs

# Data analysis goes here